[LangChain’s Router Chains and Callbacks](https://medium.com/@gil.fernandes/langchains-router-chains-and-callbacks-722524c4aa42)

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

from prompt_toolkit import HTML, prompt
import langchain.callbacks

langchain.callbacks.StdOutCallbackHandler

from FileCallbackHandler import FileCallbackHandler
from langchain.llms.bedrock import Bedrock
from pathlib import Path

file_ballback_handler = FileCallbackHandler(Path('router_chain.txt'), print_prompts=True)

In [2]:
class Config(): 
    llm = Bedrock(model_id="anthropic.claude-v2:1",  model_kwargs={'max_tokens_to_sample':200})

cfg = Config()

In [3]:
class PromptFactory():
    developer_template = """You are a very smart Python programmer. \
    You provide answers for algorithmic and computer problems in Python. \
    You explain the code in a detailed manner. \

    Here is a question:
    {input}"""

    poet_template = """You are a poet who replies to creative requests with poems in English. \
    You provide answers which are poems in the style of Lord Byron or Shakespeare. \

    Here is a question:
    {input}"""

    wiki_template = """You are a Wikipedia expert. \
    You answer common knowledge questions based on Wikipedia knowledge. \
    Your explanations are detailed and in plain English.

    Here is a question:
    {input}"""

    image_creator_template = """You create a creator of images. \
    You provide graphic representations of answers using SVG images.

    Here is a question:
    {input}"""

    legal_expert_template = """You are a UK or US legal expert. \
    You explain questions related to the UK or US legal systems in an accessible language \
    with a good number of examples.

    Here is a question:
    {input}"""



    prompt_infos = [
        {
            'name': 'python programmer',
            'description': 'Good for questions about coding and algorithms',
            'prompt_template': developer_template
        },
        {
            'name': 'poet',
            'description': 'Good for generating poems for creative questions',
            'prompt_template': poet_template
        },
        {
            'name': 'wikipedia expert',
            'description': 'Good for answering questions about general knowledge',
            'prompt_template': wiki_template
        },
        {
            'name': 'graphical artist',
            'description': 'Good for answering questions which require an image output',
            'prompt_template': image_creator_template
        },
        {
            'name': 'legal expert',
            'description': 'Good for answering questions which are related to UK or US law',
            'prompt_template': legal_expert_template
        }
    ]


In [4]:
def generate_destination_chains():
    """
    Creates a list of LLM chains with different prompt templates.
    """
    prompt_factory = PromptFactory()
    destination_chains = {}
    for p_info in prompt_factory.prompt_infos:
        name = p_info['name']
        prompt_template = p_info['prompt_template']
        chain = LLMChain(
            llm=cfg.llm, 
            prompt=PromptTemplate(template=prompt_template, input_variables=['input']))
        destination_chains[name] = chain
    default_chain = ConversationChain(llm=cfg.llm, output_key="text")
    return prompt_factory.prompt_infos, destination_chains, default_chain

In [5]:
def generate_router_chain(prompt_infos, destination_chains, default_chain):
    """
    Generats the router chains from the prompt infos.
    :param prompt_infos The prompt informations generated above.
    """
    destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
    destinations_str = '\n'.join(destinations)
    router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
    router_prompt = PromptTemplate(
        template=router_template,
        input_variables=['input'],
        output_parser=RouterOutputParser()
    )
    router_chain = LLMRouterChain.from_llm(cfg.llm, router_prompt)
    return MultiPromptChain(
        router_chain=router_chain,
        destination_chains=destination_chains,
        default_chain=default_chain,
        verbose=True,
        callbacks=[file_ballback_handler]
    )

In [6]:
prompt_infos, destination_chains, default_chain = generate_destination_chains()
chain = generate_router_chain(prompt_infos, destination_chains, default_chain)
result = chain.run('Is it ok to open a company in h1b')
print(result)
print()

/Users/tanaycho/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/tanaycho/anaconda3/envs/genai/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
legal expert: {'input': 'Is it ok to open a company in h1b visa status in the US?'}
> Finished chain.
 Sure, it is generally permissible for someone on an H-1B visa to open a company in the United States, with a few important caveats:

- The company must be set up such that the H-1B visa holder does not violate the terms of their visa status. This means they cannot be an employee of their own company, since the H-1B visa requires them to work for the petitioning employer. 

- However, the H-1B visa holder can own a company and even manage its operations and strategy. They simply cannot officially be an employee on the payroll who receives a salary. Typically they would be a passive investor or unpaid director/officer of the company.

- The company itself must comply with all normal state and federal regulations for its business operations and corporate structure. The visa status of its founder or owner does not exempt it from any requirements.